In [1]:
!pip install selenium
!pip install bs4
!pip install requests
!pip install urllib3
!pip install webdriver_manager
!pip install openpyxl
!pip install xlrd
!pip install datetime
!pip install lxml
!pip install pandas
#다 한 다음에 주피터 노트북 완전 껐다가 재 실행(이 브라우저 창만 끄지말고 아예 꺼야함)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [bs4]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [openpyxl]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [datetime]━━ 1/3 [zope.interface]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 4.7 MB/s  0:00:01a 0:00:010:00:01:01
  Using cached numpy-2.2.6-cp310-cp310-macosx_14_0_arm64.whl.metadata (62 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 6.5 MB/s  0:00:01m 6.6 MB/s eta 0:00:01
Using cached numpy-2.2.6-cp310-cp310-macosx_14_0_arm64.whl (5.3 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [pandas]━━━━━━━━━━━ 2/3 [pandas]


In [3]:
## 필요한 패키지 로딩 
import time
start_time = time.time() 
import datetime
import requests
import selenium
import lxml
import re
import warnings 
warnings.filterwarnings('ignore')

from selenium.webdriver.common.by import By
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
from openpyxl import Workbook
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from requests.packages.urllib3.util.retry import Retry
from selenium import webdriver

In [4]:
import re, json, time, datetime
from pathlib import Path
from tqdm import tqdm

import requests
from requests.adapters import HTTPAdapter, Retry
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# ---------------------------
# 설정
# ---------------------------
INPUT_JSONL = "/Users/changjin/Desktop/Workspace/lab/sac/관광지_crawaling/[6]tour_places_with_review_urls.jsonl"
OUTPUT_JSONL = "/Users/changjin/Desktop/Workspace/lab/sac/관광지_crawaling/[7]tour_places_with_reviews.jsonl"

# 페이지 스크롤 수 설정
MAX_SCROLLS = 3
REVIEW_LIMIT = 10  # 최대 리뷰 수

# ---------------------------
# 날짜를 YYYYMMDD int로
# ---------------------------
def parse_date_to_yyyymmdd(s: str, now=None):
    if not s:
        return None
    s = str(s).strip()
    if now is None:
        now = datetime.datetime.now()

    m = re.search(r'(\d{4})[.\-/](\d{1,2})[.\-/](\d{1,2})', s)
    if m:
        y, mth, d = int(m.group(1)), int(m.group(2)), int(m.group(3))
        try:
            return int(datetime.date(y, mth, d).strftime('%Y%m%d'))
        except:
            pass

    if '오늘' in s:
        return int(now.strftime('%Y%m%d'))
    if '어제' in s:
        return int((now - datetime.timedelta(days=1)).strftime('%Y%m%d'))
    m = re.search(r'(\d+)\s*일\s*전', s)
    if m:
        days = int(m.group(1))
        return int((now - datetime.timedelta(days=days)).strftime('%Y%m%d'))

    # "7.18." 같이 연도 없는 형식 → 올해로 보정
    m = re.search(r'(\d{1,2})[.]\s*(\d{1,2})[.]', s)
    if m:
        y = now.year
        mth, d = int(m.group(1)), int(m.group(2))
        try:
            return int(datetime.date(y, mth, d).strftime('%Y%m%d'))
        except:
            pass

    # 숫자 8자리만 온 경우
    if re.fullmatch(r'\d{8}', s):
        return int(s)
    return None

# ---------------------------
# place_id: URL에서 숫자
# ---------------------------
def extract_place_id(url: str):
    if not url:
        return None
    m = re.search(r'/restaurant/(\d+)', url)
    return m.group(1) if m else None

# ---------------------------
# Home: 가게명, 카테고리, 방문자/블로그 리뷰수, 설명(XtBbS)
# ---------------------------
def scrape_home(driver, place_id):
    if not place_id:
        return None, None, None, None, None
        
    home_url = f"https://m.place.naver.com/restaurant/{place_id}/home"
    try:
        driver.get(home_url)
        driver.implicitly_wait(5)

        place_name = None
        category = None
        visitor_count, blog_count = None, None
        soup = None

        try:
            WebDriverWait(driver, 6).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '#_title'))
            )
            html = driver.page_source
            soup = BeautifulSoup(html, 'lxml')

            # #_title 하위 span들: [0]=상호, [1]=카테고리 (스크린샷 기준)
            spans = soup.select('#_title span')
            if spans:
                place_name = spans[0].get_text(strip=True) if len(spans) >= 1 else None
                category = spans[1].get_text(strip=True) if len(spans) >= 2 else None

            # 보강: 클래스 기반 (상호=GHAhO, 카테고리=lnJfT)
            if not place_name:
                el = soup.select_one('#_title span.GHAhO')
                if el: place_name = el.get_text(strip=True)
            if not category:
                el = soup.select_one('#_title span.lnJfT')
                if el: category = el.get_text(strip=True)
        except:
            pass

        # 방문자/블로그 리뷰수
        try:
            if soup is None:
                soup = BeautifulSoup(driver.page_source, 'lxml')
            a_visitor = soup.select_one('a[href*="/review/visitor"]')
            if a_visitor:
                t = a_visitor.get_text(" ", strip=True)
                m = re.search(r'(\d[\d,]*)', t)
                if m: visitor_count = int(m.group(1).replace(',', ''))

            a_blog = soup.select_one('a[href*="/review/ugc"]')
            if a_blog:
                t = a_blog.get_text(" ", strip=True)
                m = re.search(r'(\d[\d,]*)', t)
                if m: blog_count = int(m.group(1).replace(',', ''))
        except:
            pass

        # 부가 설명(XtBbS)
        description = None
        try:
            if soup is None:
                soup = BeautifulSoup(driver.page_source, 'lxml')
            desc_div = soup.select_one('div.XtBbS')
            if desc_div:
                description = desc_div.get_text(" ", strip=True)
        except:
            pass

        return place_name, category, visitor_count, blog_count, description
    except Exception as e:
        print(f"❌ Home 스크래핑 오류: {e}")
        return None, None, None, None, None

# ---------------------------
# 리뷰(방문자): 추천순 클릭 후 상위 N개 (본문만 추출)
# ---------------------------
CONTENT_SELECTORS = [
    # 펼쳐진 본문("접기") 버튼 내부: 보통 여기에 전체 텍스트가 노출됨
    'a[role="button"][data-pui-click-code*="howless"]',
    # 아직 안 펼쳤을 때 버튼 내부 텍스트(간혹 본문 전체가 들어오기도 함)
    'a[role="button"][data-pui-click-code*="rvsho"]',
    # 과거 구조(예시)
    'div.pui__vn15t2 a[role="button"]',
    # 일반 div/p 폴백
    'div',
    'p',
]

def extract_review_text_from_li(li):
    for sel in CONTENT_SELECTORS:
        node = li.select_one(sel)
        if node and node.get_text(strip=True):
            txt = node.get_text(" ", strip=True)
            # 프로필/메타("리뷰 123 사진 45 …")가 들어온 경우를 걸러내기 위한 간단 휴리스틱:
            # 문장부호·조사·어절이 거의 없으면 스킵
            if re.search(r'[.!?…]|[가-힣]{2,}', txt):
                return re.sub(r'\s+', ' ', txt)
    return None

def scrape_reviews_recommended(driver, visitor_url, n=10, max_scrolls=6):
    if not visitor_url:
        return []
        
    try:
        driver.get(visitor_url)
        driver.implicitly_wait(5)

        # 추천순 클릭
        try:
            btn = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//a[@role="button"][contains(., "추천순")]'))
            )
            driver.execute_script("arguments[0].click();", btn)
            time.sleep(1.0)
        except:
            pass

        # 스크롤하면서 각 li 안의 더보기(rvsho*) 모두 클릭
        for _ in range(max_scrolls):
            try:
                more_btns = driver.find_elements(
                    By.XPATH,
                    '//a[@role="button" and contains(@data-pui-click-code,"rvsho")]'
                )
                for b in more_btns:
                    try:
                        driver.execute_script("arguments[0].click();", b)
                        time.sleep(0.05)
                    except:
                        pass
            except:
                pass
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(0.6)

        soup = BeautifulSoup(driver.page_source, 'lxml')
        items = soup.select('ul#_review_list > li')
        reviews = []

        for li in items:
            # 본문만 가져오기
            text = extract_review_text_from_li(li)

            # 시간
            t_el = li.select_one('time[aria-hidden="true"]') or li.find('time')
            t_txt = t_el.get_text(strip=True) if t_el else None
            t_int = parse_date_to_yyyymmdd(t_txt)

            if text:
                reviews.append({"time": t_int, "text": text})
            if len(reviews) >= n:
                break

        return reviews
    except Exception as e:
        print(f"❌ 리뷰 스크래핑 오류: {e}")
        return []

def setup_driver():
    """Chrome 드라이버 설정"""
    options = webdriver.ChromeOptions()
    options.add_argument('window-size=1920x1080')
    options.add_argument('--force-device-scale-factor=0.4')
    options.add_argument("disable-gpu")
    # 헤드리스 모드 (선택사항)
    # options.add_argument('--headless')
    
    return webdriver.Chrome(options=options)

def process_jsonl():
    """JSONL 파일 처리"""
    
    # 입력 파일 읽기
    records = []
    with open(INPUT_JSONL, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                records.append(json.loads(line))
    
    print(f"📊 총 처리할 항목: {len(records)}개")
    
    processed = 0
    success = 0
    failed = 0
    
    # 출력 파일 열기
    with open(OUTPUT_JSONL, 'w', encoding='utf-8') as outfile:
        
        for i, record in enumerate(tqdm(records, desc="🔍 리뷰 크롤링 중")):
            driver = None
            
            try:
                place_name = record.get('place_name', 'Unknown')
                place_id = record.get('place_id')
                url = record.get('url', '')
                
                print(f"\n📍 처리중 ({i+1}/{len(records)}): {place_name} (ID: {place_id})")
                
                # place_id가 없으면 URL에서 추출 시도
                if not place_id:
                    place_id = extract_place_id(url)
                
                if not place_id:
                    print("❌ place_id를 찾을 수 없습니다")
                    outfile.write(json.dumps(record, ensure_ascii=False) + '\n')
                    failed += 1
                    continue
                
                # 드라이버 설정
                driver = setup_driver()
                driver.implicitly_wait(5)
                
                # Home 페이지에서 정보 수집
                p_name, category, v_cnt, b_cnt, description = scrape_home(driver, place_id)
                
                # 기존 값 업데이트
                if category:
                    record['sub_category'] = category
                    print(f"✅ sub_category: {category}")
                
                if v_cnt is not None:
                    record['visiter_review_count'] = int(v_cnt)
                    print(f"✅ visiter_review_count: {v_cnt}")
                
                if b_cnt is not None:
                    record['blog_review_count'] = int(b_cnt)
                    print(f"✅ blog_review_count: {b_cnt}")
                
                # 리뷰 수집
                if url:
                    reviews = scrape_reviews_recommended(
                        driver, url, n=REVIEW_LIMIT, max_scrolls=MAX_SCROLLS
                    )
                    record['reviews_attraction'] = reviews
                    print(f"✅ reviews_attraction: {len(reviews)}개 수집")
                else:
                    print("❌ URL이 없어 리뷰 수집 불가")
                
                success += 1
                
            except Exception as e:
                print(f"❌ 처리 오류: {e}")
                failed += 1
            
            finally:
                if driver:
                    try:
                        driver.quit()
                    except:
                        pass
                
                # 레코드 저장
                outfile.write(json.dumps(record, ensure_ascii=False) + '\n')
                processed += 1
                
                # 잠시 대기 (차단 방지)
                time.sleep(1)
    
    print("\n" + "="*60)
    print("🎉 크롤링 완료!")
    print(f"📊 결과 통계:")
    print(f"   - 총 처리: {processed}개")
    print(f"   - 성공: {success}개")
    print(f"   - 실패: {failed}개")
    print(f"📁 저장 위치: {OUTPUT_JSONL}")
    print("="*60)

def test_single_item():
    """단일 항목 테스트"""
    with open(INPUT_JSONL, 'r', encoding='utf-8') as f:
        first_line = f.readline()
        if first_line.strip():
            record = json.loads(first_line)
            
            print("🧪 테스트 항목:")
            print(f"   place_name: {record.get('place_name')}")
            print(f"   place_id: {record.get('place_id')}")
            print(f"   url: {record.get('url')}")
            
            driver = setup_driver()
            
            try:
                place_id = record.get('place_id')
                url = record.get('url')
                
                if not place_id:
                    place_id = extract_place_id(url)
                
                if place_id:
                    print(f"\n🏠 Home 정보 수집 중...")
                    p_name, category, v_cnt, b_cnt, desc = scrape_home(driver, place_id)
                    print(f"   카테고리: {category}")
                    print(f"   방문자 리뷰: {v_cnt}")
                    print(f"   블로그 리뷰: {b_cnt}")
                    
                    print(f"\n📝 리뷰 수집 중...")
                    reviews = scrape_reviews_recommended(driver, url, n=3, max_scrolls=2)
                    print(f"   수집된 리뷰: {len(reviews)}개")
                    for i, r in enumerate(reviews[:2]):
                        print(f"   리뷰 {i+1}: {r['text'][:50]}...")
                
            finally:
                driver.quit()

if __name__ == "__main__":
    print("🚀 JSONL 네이버 플레이스 리뷰 크롤러")
    print("="*60)
    
    import sys
    if len(sys.argv) > 1 and sys.argv[1] == "test":
        test_single_item()
    else:
        process_jsonl()

🚀 JSONL 네이버 플레이스 리뷰 크롤러
📊 총 처리할 항목: 152개


🔍 리뷰 크롤링 중:   0%|          | 0/152 [00:00<?, ?it/s]


📍 처리중 (1/152): 국립대관령자연휴양림 (ID: 11491009)
✅ sub_category: 휴양림,산림욕장
✅ visiter_review_count: 771
✅ blog_review_count: 550
✅ reviews_attraction: 10개 수집


🔍 리뷰 크롤링 중:   1%|          | 1/152 [00:27<1:08:22, 27.17s/it]


📍 처리중 (2/152): 경포도립공원 (ID: 13300191)
✅ sub_category: 도립공원
✅ visiter_review_count: 26
✅ blog_review_count: 69
✅ reviews_attraction: 10개 수집


...


📍 처리중 (151/152): 소금강 돌박물관 (ID: 15374325)
✅ sub_category: 박물관
✅ visiter_review_count: 1
✅ blog_review_count: 7
✅ reviews_attraction: 0개 수집


🔍 리뷰 크롤링 중:  99%|█████████▉| 151/152 [35:39<00:16, 16.13s/it]


📍 처리중 (152/152): 갈골한과체험전시관 (ID: 15704484)
✅ sub_category: 박람회,전시회
✅ visiter_review_count: 18
✅ blog_review_count: 6
✅ reviews_attraction: 10개 수집


🔍 리뷰 크롤링 중: 100%|██████████| 152/152 [35:49<00:00, 14.14s/it]


🎉 크롤링 완료!
📊 결과 통계:
   - 총 처리: 152개
   - 성공: 152개
   - 실패: 0개
📁 저장 위치: /Users/changjin/Desktop/Workspace/lab/sac/관광지_crawaling/[7]tour_places_with_reviews.jsonl
